In [1]:
#generation debugging 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
import networkx as nx
import rdkit
import math, random, sys
import numpy as np
import argparse
import os
from tqdm.auto import tqdm
import pickle
from hgraph import *
from hgraph.inc_graph import *
from hgraph.encoder import *
import matplotlib.pyplot as plt

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vocab = '/home/marcase/hgraph2graph/data/chembl/cyclic_vocab_new.txt'
vocab = [x.strip("\r\n ").split() for x in open(vocab)]
vocab = PairVocab(vocab)

In [3]:
class Args:
    train = 'cyclic_preprocessed_new/'
    vocab = vocab
    save_dir = 'test/'
    atom_vocab = common_atom_vocab
    load_model = None
    seed = 7
    rnn_type = 'LSTM'
    hidden_size=125
    embed_size=250
    batch_size=1
    latent_size=32
    depthT=15
    depthG=15
    diterT=1
    diterG=3
    dropout=0.0
    lr = 1e-3
    clip_norm=5.0
    step_beta=0.001
    max_beta=1.0
    warmup=10000
    kl_anneal_iter=2000
    epoch=20
    anneal_rate=0.9
    anneal_iter=25000
    print_iter=50
    save_iter=5000
    model = '/home/marcase/hgraph2graph/ckpt/cyclic_new2/model.ckpt.140000'
    load_model = True
    nsample = 1
    
args=Args()

model = HierVAE(args).cuda()

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [4]:
model = HierVAE(args).cuda()

model.load_state_dict(torch.load(args.model)[0])
model.eval()



/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


HierVAE(
  (encoder): HierMPNEncoder(
    (E_c): Sequential(
      (0): Embedding(23, 250)
      (1): Dropout(p=0.0, inplace=False)
    )
    (E_i): Sequential(
      (0): Embedding(28, 250)
      (1): Dropout(p=0.0, inplace=False)
    )
    (W_c): Sequential(
      (0): Linear(in_features=375, out_features=125, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.0, inplace=False)
    )
    (W_i): Sequential(
      (0): Linear(in_features=375, out_features=125, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.0, inplace=False)
    )
    (W_root): Sequential(
      (0): Linear(in_features=250, out_features=125, bias=True)
      (1): Tanh()
    )
    (tree_encoder): MPNEncoder(
      (W_o): Sequential(
        (0): Linear(in_features=250, out_features=125, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.0, inplace=False)
      )
      (rnn): LSTM(
        (W_i): Sequential(
          (0): Linear(in_features=300, out_features=125, bias=True)
          (1): Sigmoid()
        )
 

In [ ]:
with torch.no_grad():
    for _ in tqdm(range(args.nsample // args.batch_size)):
        smiles_list = model.sample(args.batch_size, greedy=True)
        for _,smiles in enumerate(smiles_list):
            print(smiles)

  0%|          | 0/1 [00:00<?, ?it/s]